# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
city_weather = pd.read_csv("../WeatherPy/city_weather.csv")
city_weather = city_weather[['City', 'Coordinates', 'Latitude', 'Longitude',
       'Temperature', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country',
       'Date']]
city_weather.head(5)

,City,Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,new norfolk,"147.06, -42.78",-42.78,147.06,35.58,80,5,4.70,AU,1598291898
1,nikolskoye,"30.79, 59.7",59.70,30.79,61.63,77,40,4.47,RU,1598291898
2,lasem,"111.45, -6.69",-6.69,111.45,75.43,76,31,7.05,ID,1598291899
3,ushuaia,"-68.3, -54.8",-54.80,-68.30,35.60,74,75,16.11,AR,1598291899
4,mataura,"168.86, -46.19",-46.19,168.86,46.99,86,95,13.00,NZ,1598291899


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [41]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
# Create a list containing coordinates
lat = city_weather["Latitude"]
lon = city_weather["Longitude"]

coordinates = city_weather[["Latitude", "Longitude"]]

#Customize the size of the figure

figure_layout = {
    'width': '750px',
    'height': '650px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
# Assign the marker layer to a variable
#markers = gmaps.marker_layer(coordinates)  #, fill_color = "red", stroke_color = "red", scale = 2)

#Add heatmap layer
locations = city_weather[["Latitude", "Longitude"]].astype(float)
humidity = city_weather["Humidity"].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

# Add the layer to the map
#fig.add_layer(markers)
fig

#coordinates.head(5)

Figure(layout=FigureLayout(border='1px solid black', height='650px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
